In [ ]:
from toolbox.audio2img import audio_to_img

path= 'Data/test/soundscape_29201.ogg'
img= audio_to_img(path,
                  period= 5,
                  sr= 32000,
                  n_mels= 128,
                  fmin = 20,
                  fmax = 16000)
img.max()

# make audio image

In [ ]:
import glob
from PIL import Image
from tqdm.auto import tqdm
import os
import numpy as np
from toolbox.audio2img import audio_to_img
import soundfile as sf
import librosa

c= 0
paths= glob.glob('Data/train_origin/**/*ogg', recursive=True)
for path in tqdm(paths):
    name= path.split('\\')[-1].replace('ogg', '')
    
    new_path= path.replace('train_origin', 'train_img_256')
    new_path= new_path.split('\\')[:-1]
    new_path= '/'.join(new_path)
    os.makedirs(new_path, exist_ok=True)
    
    imgs= audio_to_img(path,
                      period= 5,
                      sr= 32000,
                      n_mels= 256,
                      n_fft= 2048,
                      hop_length= 512,
                      fmin = 16,
                      fmax = 16386)

    if len(imgs)!=1: imgs= imgs[:-1]
    for i in range(len(imgs)):
        img= imgs[i]
        im= Image.fromarray(img.astype(np.uint8))
        im.save(f'{new_path}/{name}_{c}.png')
        c+= 1
        
#     sr= 32000
#     period= 5
#     data, sr = librosa.load(path, sr= sr)
#     max_sec= int( len(data)//sr )+1
#     datas = [ data[i * sr: (i+period) * sr] for i in range(0, max_sec, period) ]
#     if len(datas[-1])<sr*period: datas[-1]= list(datas[-1]) + [0]*( sr*period-len(datas[-1]) )
    
#     for i in range(len(datas)):
#         data= datas[i]
#         sf.write(f'{new_path}/{name}_{c}.ogg', data, sr, format='ogg', subtype='vorbis')
#         c+= 1

In [ ]:
import pandas as pd
from io import StringIO
from csv import writer 

paths= glob.glob('Data/train_img_ex1/**/*png', recursive=True)
label_name= os.listdir('Data/train_img_ex1')

df = StringIO()
csv_writer = writer(df)
csv_writer.writerow(['image_path', 'label', 'label_name', 'group'])

for i, path in enumerate(tqdm(paths)):
    group= path.split('\\')[-1].split('_')[0]
    label= path.split('\\')[-2]
    label_num= label_name.index(label)
    csv_writer.writerow([path, label_num, label, group])
    
df.seek(0)
df= pd.read_csv(df)
df

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold

group_id= df['group']
kf = GroupKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(group_id, group_id, group_id)):
    for indx in tqdm(test_index):
        df.loc[indx, 'fold']= i
df

In [ ]:
label= df['label'].unique().tolist()
for l in tqdm(label):
    temp_df= df[df['label']==l]
    group= temp_df['group'].unique().tolist()
    if len(group)<5:
        df.loc[ df['label']==l, 'fold' ]= -1
df

# Model Soup

In [6]:
import torch
import torch.nn as nn

class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
    def forward(self, image):
        x = self.model(image)
        return x
    
def get_soup(state_dicts, alphal):
    sd = {k : state_dicts[0][k].clone() * alphal[0] for k in state_dicts[0].keys()}
    for i in range(1, len(state_dicts)):
        for k in state_dicts[i].keys():
            sd[k] = sd[k] + state_dicts[i][k].clone() * alphal[i]
    return sd

    
model_path=[
    'test_model/effb0_imgsz128_PL1/model_soup.pth',
    'test_model/effb0_imgsz128_PL2/model_soup.pth',
#     'test_model/convnextv2_tiny_imgsz128_PL3/cv2_best.pth',
#     'test_model/convnextv2_tiny_imgsz128_PL3/cv3_best.pth',
#     'test_model/convnextv2_tiny_imgsz128_PL3/cv4_best.pth',
]
model_weights= [torch.load(path).state_dict() for path in model_path]
# alphal= [1/len(model_path)]*len(model_path)
alphal= [0.25, 0.75]

soup= get_soup(model_weights, alphal)
model_soup= torch.load(model_path[0])
model_soup.load_state_dict(soup)
torch.save(model_soup, 'test_model/model_soup.pth')
print('successful')

successful


# PTH2Torchscript

In [7]:
import torch
import torch.nn as nn

class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        
    def forward(self, image):
        x = self.model(image)
        return x

# path= 'train_model/cv0_best.pth'
path= 'test_model/model_soup.pth'
# path= 'test_model/effb0_imgsz192_PL3/cv4_best.pth'

example = torch.rand(1, 3, 128, 313)
# example = torch.rand(1, 3, 192, 470)

model= torch.load(path).cpu().eval()
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(path.replace('pth', 'ts'))